In [27]:
import pandas as pd
import numpy as np

import spacy
from spacy.training.example import Example
from spacy.tokens import DocBin, Span
from spacy import displacy

In [2]:
DATA_FILE = 'data/train_data.csv'
DATA_TEST = 'data/gt_test.csv'

# Подготовка данных

In [7]:
data = pd.read_csv(DATA_FILE)
print(data.head())
print(data.info())

                                      processed_text  \
0  аа союзная тридцать пять дробь один лариса сое...   
1  аа приложение мне показывает к оплате у меня п...   
2  да лисное по призрак лишнее ну почему иду пять...   
3  а что добрый день NAME у меня пришел какой то ...   
4  у меня западный с утра да да еще да да самый в...   

                             target_labels_positions  
0                                                 {}  
1                                                 {}  
2  {'I-value': [140], 'B-value': [139], 'B-discou...  
3                               {'B-discount': [12]}  
4                                                 {}  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3399 entries, 0 to 3398
Data columns (total 2 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   processed_text           3399 non-null   object
 1   target_labels_positions  3399 non-null   object
dtypes: object(

**датафрейм содержаший наблюдения без скидок и размеченый**

In [8]:
no_data = data[~data['processed_text'].str.contains('скид', case=False)]

In [9]:
no_data

,processed_text,target_labels_positions
1,аа приложение мне показывает к оплате у меня п...,{}
5,NAME ну а по поводу ипотеки по моему ну москов...,{}
6,да пока здесь минутку я уразил жду врача да ин...,{}
8,а хата амально позвонили NAME да если можно ес...,{}
11,добрый день мне не звонили я перезваниваю да д...,{}
...,...,...
3385,нет добрый день NAME зовут подскажите пожалуйс...,{}
3389,здравствуйте да подскажите пожалуйста у вас бе...,{}
3396,целенаправлен на голосовой почтовый ящик вы мо...,{}
3397,NAME зовут NAME я хотела уточнить ээ своего ме...,{}


In [10]:
no_data['target_labels_positions'].value_counts()

{}    1691
Name: target_labels_positions, dtype: int64

**датафрейм с наблюдениями содержащий скидки и размеченый**

In [18]:
yes_data = data[data['target_labels_positions'] != '{}']
yes_data

,processed_text,target_labels_positions
2,да лисное по призрак лишнее ну почему иду пять...,"{'I-value': [140], 'B-value': [139], 'B-discou..."
3,а что добрый день NAME у меня пришел какой то ...,{'B-discount': [12]}
9,NAME добрый день NAME зовут а NAME сейчас назе...,"{'I-value': [117], 'B-value': [116], 'B-discou..."
10,а здравствуйте NAME зовут NAME я по поводу пок...,"{'I-value': [374], 'B-value': [373], 'B-discou..."
12,добрый день имя клиентов NAME в жилом комплекс...,{'B-discount': [213]}
...,...,...
3387,здравствуйте я хотела бы проконсультировать у ...,{'B-discount': [350]}
3390,все привет заданочка все б ты меня не понял я ...,"{'B-value': [24], 'B-discount': [12]}"
3392,да здравствуйте да ну просто давай ришат меня ...,"{'I-value': [227], 'B-value': [226], 'B-discou..."
3393,NAME а то сорвочно да два угу а ну да то ну до...,"{'I-value': [243, 244], 'B-value': [242], 'B-d..."


In [19]:
yes_data['target_labels_positions'].value_counts()

{'B-discount': [30]}                                                      3
{'I-value': [210], 'B-value': [209], 'B-discount': [208]}                 2
{'I-value': [91], 'B-value': [90], 'B-discount': [89]}                    2
{'I-value': [65], 'B-value': [64], 'B-discount': [60]}                    2
{'I-value': [234], 'B-value': [233], 'B-discount': [232]}                 2
                                                                         ..
{'I-value': [94], 'B-value': [93], 'B-discount': [92]}                    1
{'I-value': [272], 'B-value': [271], 'B-discount': [270]}                 1
{'I-value': [93, 225], 'B-value': [92, 224], 'B-discount': [91, 223]}     1
{'I-value': [22, 23, 94], 'B-value': [21, 93], 'B-discount': [20, 81]}    1
{'I-value': [224, 225], 'B-value': [223], 'B-discount': [222]}            1
Name: target_labels_positions, Length: 481, dtype: int64

**датафрейм содержащий неразмеченные данные**

In [23]:
unique_data = data[~(data.isin(no_data).all(1) | data.isin(yes_data).all(1))]
unique_data

,processed_text,target_labels_positions
0,аа союзная тридцать пять дробь один лариса сое...,{}
4,у меня западный с утра да да еще да да самый в...,{}
7,да спа спал так подобное десять пар не сюда ну...,{}
16,здравствуйте я приеду записала на консультацию...,{}
17,здравствуйте девушка алиса я хотела бы спросит...,{}
...,...,...
3383,сколько еле с утра алло да на то ли пора как т...,{}
3384,свяжите пожалуйста с реактором по креветская с...,{}
3388,что хотели связаться с менеджером интересует н...,{}
3391,добрый день NAME звонок это вита живой комплек...,{}


In [74]:
unique_data = unique_data.reset_index(drop=True)
unique_data

,processed_text,target_labels_positions
0,аа союзная тридцать пять дробь один лариса сое...,{}
1,у меня западный с утра да да еще да да самый в...,{}
2,да спа спал так подобное десять пар не сюда ну...,{}
3,здравствуйте я приеду записала на консультацию...,{}
4,здравствуйте девушка алиса я хотела бы спросит...,{}
...,...,...
1200,сколько еле с утра алло да на то ли пора как т...,{}
1201,свяжите пожалуйста с реактором по креветская с...,{}
1202,что хотели связаться с менеджером интересует н...,{}
1203,добрый день NAME звонок это вита живой комплек...,{}


In [76]:
unique_data.to_csv('data_v2/anlabled.csv', index=False)

**создаем учебный датафрейм из данных со скидкой и данных безскидки в соотношении 1 к 1**

In [28]:
# Генерируем 503 псевдослучайных числа в диапазоне длины исходного датафрейма
indices = np.random.choice(len(no_data), size=503, replace=False)

# Создаем новый датафрейм с выбранными строками
no_data_sliced = no_data.iloc[indices]

In [29]:
no_data_sliced

,processed_text,target_labels_positions
362,ну здравствуйте я не чудо я у вас котти роста ...,{}
3076,что есть по поводу квартира ради двухкомнат не...,{}
2656,NAME мы меня зовут NAME а мы как то до этого о...,{}
821,пишу тебе сколько он там написали сама направи...,{}
3219,добрый день девушка по поводу помещения на кир...,{}
...,...,...
3248,NAME добрый день это я из агентство я риэлтор ...,{}
900,ээ здравствуйте девушка я вот подведение звоню...,{}
3238,здравствуйте меня зовут лера я бы хотела узнат...,{}
2991,да сандро я отложил там студия аа два раза кре...,{}


In [36]:
train_df = yes_data.append(no_data_sliced)
train_df

C:\Users\CheshirSmile\AppData\Local\Temp\ipykernel_14624\1695018391.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_df = yes_data.append(no_data_sliced)


,processed_text,target_labels_positions
2,да лисное по призрак лишнее ну почему иду пять...,"{'I-value': [140], 'B-value': [139], 'B-discou..."
3,а что добрый день NAME у меня пришел какой то ...,{'B-discount': [12]}
9,NAME добрый день NAME зовут а NAME сейчас назе...,"{'I-value': [117], 'B-value': [116], 'B-discou..."
10,а здравствуйте NAME зовут NAME я по поводу пок...,"{'I-value': [374], 'B-value': [373], 'B-discou..."
12,добрый день имя клиентов NAME в жилом комплекс...,{'B-discount': [213]}
...,...,...
3248,NAME добрый день это я из агентство я риэлтор ...,{}
900,ээ здравствуйте девушка я вот подведение звоню...,{}
3238,здравствуйте меня зовут лера я бы хотела узнат...,{}
2991,да сандро я отложил там студия аа два раза кре...,{}


In [40]:
shuffled_train_df = train_df.sample(frac=1)
shuffled_train_df

,processed_text,target_labels_positions
1067,NAME я у вас на звание нашла объявление двести...,{}
2760,добрый вечер далее ээ мы звонили мне несколько...,"{'I-value': [274], 'B-value': [273], 'B-discou..."
3319,вопрос по кодовой NAME нет что для себя закал ...,"{'I-value': [21], 'B-value': [20], 'B-discount..."
2899,здравствуйте я насчет квартиры первомайская со...,"{'I-value': [304, 305], 'B-value': [303], 'B-d..."
1747,удень а я хотел бы узнать вот по поводу заявки...,{}
...,...,...
1924,да добрый день NAME вам нужно по поводу подпис...,{}
3089,NAME вы не можете говорить мы вот сейчас втб х...,{}
3025,здравствуйте NAME по воскресеньям у вас продаж...,"{'I-value': [223], 'B-value': [222], 'B-discou..."
1942,NAME здравствуйте меня указалось подскажите по...,"{'B-value': [93], 'B-discount': [91]}"


# преобразуем данные

In [41]:
# Функция для преобразования данных в формат SpaCy
def convert_data_to_spacy_format(data):
    nlp = spacy.blank("ru")  # Создаем пустую модель SpaCy для русского языка
    db = DocBin()

    for index, row in data.iterrows():
        text = row['processed_text']
        labels = eval(row['target_labels_positions'])

        doc = nlp(text)
        ents = []

        for label, positions in labels.items():
            for start in positions:
                end = start + 1  # Предполагаем, что метка охватывает один токен (нужно уточнить)
                span = Span(doc, start, end,  label=label)                
                if span is not None:
                    ents.append(span)

        doc.ents = ents
        db.add(doc)

    return db

# Преобразуем данные
db = convert_data_to_spacy_format(shuffled_train_df)

# Сохраним преобразованные данные
db.to_disk("data_v2/training_data.spacy")

# Обучаем модель

In [44]:
!python -m spacy train config.cfg --output ./model_4 --paths.train ./data_v2/training_data.spacy --paths.dev ./data_v2/training_data.spacy

✔ Created output directory: model_4
ℹ Saving to output directory: model_4
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     61.07    0.00    0.00    0.00    0.00
  0     200         40.85   1606.18   37.28   49.79   29.79    0.37
  0     400        188.70    775.96   54.76   54.70   54.83    0.55
  0     600        412.53   1058.22   58.89   56.76   61.19    0.59
  0     800        105.85    405.15   58.04   62.84   53.93    0.58
  1    1000        142.00    387.88   56.30   68.14   47.97    0.56
  1    1200        274.25    583.45   56.85   65.75   50.08    0.57
  1    1400        267.38    471.23   57.62   70.59   48.67   

# ТЕСТ

In [113]:
# Загружаем тестовые данные
test_db = DocBin().from_disk("data/test_data.spacy")
nlp = spacy.load('model_4/model-best')
test_docs = list(test_db.get_docs(nlp.vocab))

# Оцениваем модель
examples = []
for doc in test_docs:
    pred_doc = nlp(doc.text)
    example = Example(pred_doc, doc)
    examples.append(example)

scorer = nlp.evaluate(examples)
print(scorer)

{'token_acc': 1.0, 'token_p': 1.0, 'token_r': 1.0, 'token_f': 1.0, 'ents_p': 0.512280701754386, 'ents_r': 1.0, 'ents_f': 0.6774941995359629, 'ents_per_type': {'B-discount': {'p': 0.43795620437956206, 'r': 1.0, 'f': 0.6091370558375635}, 'B-value': {'p': 0.6532258064516129, 'r': 1.0, 'f': 0.7902439024390243}, 'I-value': {'p': 0.5290697674418605, 'r': 1.0, 'f': 0.6920152091254753}}, 'speed': 45454.95592286359}


In [104]:
def ent_position(doc):
    entity_data = {}

    for ent in doc.ents:
        label = ent.label_
        entity_tokens = [token.i for token in ent]
        # Преобразование начального и конечного токена в границы
        adjusted_start = max(0, entity_tokens[0])
        adjusted_end = min(len(doc), entity_tokens[-1] + 1)
        
        # Объединяем перекрывающие типы для каждого объекта
        merged_segments = []
        current_segment = []
        segment_start = None
        segment_end = None
        for i in entity_tokens:
            if not segment_start or i > segment_end+1:
                if segment_start:
                    merged_segments.append((segment_start, segment_end))
                segment_start = i
                segment_end = i
            elif i <= segment_end:
                segment_end += 1
                
        final_entity_tokens = [adjusted_start] + sorted(merged_segments, key=lambda x: x[0]) #+ [adjusted_end]
        entity_data[label] = final_entity_tokens
    return entity_data

In [94]:
def model_tester(model, text):
    nlp = spacy.load(model)
    doc = nlp(text)

    for ent in doc.ents:
        print(ent.text, ent.label_)
        
    for ent in doc.ents:
        entity_region = doc[ent.start-2 : ent.end+2] if len(doc) >= ent.end+2 else doc[ent.start-2:]
        displacy.render(entity_region, style='ent', jupyter=True)

    print(spacy.displacy.render(doc, style="ent", jupyter=True))
    print(ent_position(doc))

In [92]:
TEST_MODEL = 'model_4/model-best'
#'data/_tmp/v_base/ner_model'

In [50]:
Test_2 = 'здравствуйте меня долбили я звоню вам аа по партнерской программе да то есть мы работаю с вами через компанию фэнмаркет\
    я бы хотела зафиксировать клиента вам на объекты должны по делам приехать с клиен как то можно через двадцать да наверное смотрите\
        мы едем в платье и пятницкие луга там просто мы обычно угу хорошо нет нет она на своем автомобиле единственное что да смс по маршруту\
            да и по ивакиной и по пятницким лугам и там наверное будет информат по скидке да один процент пока только этот клиент расскажите\
                пожалуйста как вам напрямую дозваниваться потому что очень сложно ну чтобы я сразу обычно попадала таким вопросом а не на банальный\
                    ваш номер телефона к вам колл центр ну чтобы таким образом зафиксировали есть угу они соединяются там дальше я получается ольге да\
                        сейчас придет уведомление позвонил что вы нам зафиксировали скидку за раннее бронирование и направили смс клиенту маршрут да да\
                            здравствуйте я хотела бы зафиксировать приезд клиентов на два жк да мы их зафиксировали если не пакета мы бы хотели предупредить\
                                что будет сегодня приезд возил продаж и мы обычно еще просим собисовать дополнительную скидку там за срочное бронирование у вас\
                                    еще работает ага и если можно еще на один да раньше было два аа сейчас один пусть хотя бы один и если можно направить смс клиентке\
                                        с маршрутом как добраться ну с подтверждения да на автомобиль меня заулили так значит клиентка NAME с телефона NAME восемь пятьсот\
                                            семьдесят семь девяносто девять двадцать пять это будет квартал да человека и пятницкий луга а подскажите пожалуйста вот пятницкий\
                                                луга там сейчас действует ээ система ээ по ипотеке рубль на месте у угу не на всех проектах просто вот эта программа реализована\
                                                    ну хорошо на месяц тогда выгу да угу сегодня в ближайшее время на двенадцатое может быть она подъедет подождет в режим очереди\
                                                        рядом по фактуприкинь копании самолете как могу к вам брать меня зовут NAME вам нужен один восприняли пароль так машин да\
                                                            тогда я просто еще доехать да я так еще будет один или два объекта записали ага продажкал центр тоже нога дай я я поняла\
                                                                вас вы работаете как аген номер телефона двадцать шесть двадцать семь актуален я брат давайте на линии спасибо за ожидание\
                                                                    перевожу вас продаж налить пожалуйста не за уникаль ага так ну скажем так у нас она надеваю она называется ээ если выезда\
                                                                        и на быстрый видит было два сейчас за немножечко да налично там скажите пожалуйста к вам да я слушаю опер два случись\
                                                                            уточнитов много ипотеки там ставка то аня и не могу точно подтвердить ну хорошо интеллект если да на ивакина на имперую встречу дала проезжет'

In [106]:
model_tester(TEST_MODEL,Test_2)

скидке B-discount
один B-value
процент I-value


None
{'B-discount': [84], 'B-value': [86], 'I-value': [87]}


In [53]:
Test_3 = 'но так сразу алло добрый день я только что разговор там с какой то девушкой прервал звонок по приобретению\
    в кредит NAME и где то она осташливала там что то еще я не знаю что она предлагала какие то вариант мне нужны ипотечного\
        вообще менеджера есть мост вообще деньги с ними можно буду перезванивать если о неспрашиваю трубку на себя здравствуйте\
            я хотел взять квартиру в ипотеку именно в семье раз рассматриваю внуково новое там астафьев где там еще в ADDRESS да в\
                ADDRESS ну семейная ипотека до двенадцати миллионов семейная ипотека до двенадцати мне больше не дают ну которой двенадцать\
                    миллионов списывается такая комнаты а не понял сейчас подряд я сказал западные ну да у нас семейная ипотека и ятека поэтому\
                        там семьдесят процентов должно быть да можно у вас там у вас там скидка еще девять продали ну что у меня есть продажа ага\
                            а вот это западное ну уже что еще не построено а вот вы мне нужен отечный менеджер но вы да мне тоже так девушка сказала\
                                но со мной связался опять мой менеджер по продажам а мне нужен ипотеч да передачадобрый день группы самолет меня зовут\
                                    агер как могу обращаться к вам какой вопрос да добрый день группы да а по какому вопросу обращаюсь какой живой комплекс\
                                        так я удачи сейчас снова буду соединять сотру отдел продаж ответить любой свободный менеджер он вас также проконсультирует\
                                            именно по ипотеке вас проконсультирует верно а подскажите как могу обратиться к вам мы рад очень при я понял да мы поэтому\
                                                вам и перезвониваем звонок поступил и чтобы вас верно соединить и направить марата подскажите для себя интересуетесь или\
                                                    как агент обращаетесь и номер вашего телефона тридцать два пятнадцать заканчивается актуален верно оставайтесь пожалуйста\
                                                        на линии соединяю вас не кладите трубку самолет меня зовут в каком жилом комплексе много прокого NAME оставьтеа ADDRESS ADDRESS\
                                                            ма север город ну тут у нас у меня неважно сложно все многое квартал аа в квартале западный двух комнатное от десяти миллионов девяти\
                                                                старого больше там двенадцать сейчас кипятка только прислали на объект передайте информацию офис продаж угу информацию'

# Готовим сабмит

In [ ]:
def add_ner_positions(input_csv, output_csv, model):
    
    # загружаем данные
    df = pd.read_csv(input_csv)
    
    def ent_position(doc):
        entity_data = {}

        for ent in doc.ents:
            label = ent.label_
            entity_tokens = [token.i for token in ent]
            # Преобразование начального и конечного токена в границы
            adjusted_start = max(0, entity_tokens[0])
            adjusted_end = min(len(doc), entity_tokens[-1] + 1)
            
            # Объединяем перекрывающие типы для каждого объекта
            merged_segments = []
            current_segment = []
            segment_start = None
            segment_end = None
            for i in entity_tokens:
                if not segment_start or i > segment_end+1:
                    if segment_start:
                        merged_segments.append((segment_start, segment_end))
                    segment_start = i
                    segment_end = i
                elif i <= segment_end:
                    segment_end += 1
                    
            final_entity_tokens = [adjusted_start] + sorted(merged_segments, key=lambda x: x[0]) #+ [adjusted_end]
            entity_data[label] = final_entity_tokens
        return entity_data
    
        # Перебираем строки в DataFrame, извлекая именованные объекты и их позиции.
    results = []
    temp_dict = dict
    for _, row in df.iterrows():
        text = row["processed_text"]
        doc = model(text)
        
        if len(doc.ents) == 'О':
            # Возвращаем пустой словарь если метки не найдены
            label = {}
        else:
            for ent in doc.ents:
                label = ent_position(doc)
        
        # Сохраняем извлеченную информацию во временный словарь
        temp_dict = {k: v for k, v in row.items()}
        temp_dict["labels_position"] = label
        #temp_dict["chars"] = str(label) # Convert the label to a string before saving it

        # Добавить обновленную строку в список результатов
        results.append(temp_dict)

    # Создаем новый DataFrame из обработанных данных
    output_df = pd.DataFrame(results)
    
    output_df.to_csv(output_csv, index=False)


In [ ]:
add_ner_positions(DATA_TEST, "output_test.csv", spacy.load(TEST_MODEL))

In [ ]:
output = pd.read_csv('output_test.csv')
output = output.drop('label', axis=1)

In [ ]:
def transform_text(doc):
    """Labels tokens based on their relation to entities."""
    labeled_tokens = []
    for token in doc:
        if token.ent_type_ != '':
            labeled_tokens.append(token.ent_type_)
            for _ in range(token.i, token.head.i):
                labeled_tokens.append(token.ent_type_)
        else:
            labeled_tokens.append('O')
    return labeled_tokens

transformed_texts = []
for _, row in output.iterrows():
    text = row["processed_text"]
    doc = model(text)

    transformed_text = transform_text(doc)

    temp_dict = row.to_dict()
    temp_dict["label"] = transformed_text

    transformed_texts.append(temp_dict)

submission = pd.DataFrame(transformed_texts)

submission.to_csv('submission.csv', index=False)